1. Creating New Features From "PassengerId" Feature

- We know that each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number of people within the group.
- So we can create a new feature Group_Size which will indicate total number of members present in each group.
- We can create one more new feature Travelling Solo indicating whether the passenger is travelling solo or in a group.

In [2]:
#import required packages 

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
#read in the train and test datasets

train_raw=pd.read_csv('train_raw.csv',delimiter=',')

test_raw=pd.read_csv('test_raw.csv',delimiter=',')

In [3]:
def passengerid_new_features(df):
    
    #Splitting Group and Member values from "PassengerId" column.
    df["Group"] = df["PassengerId"].apply(lambda x: x.split("_")[0])
    df["Member"] =df["PassengerId"].apply(lambda x: x.split("_")[1])
    
    #Grouping the "Group" feature with respect to "member" feature to check which group is travelling with how many members
    x = df.groupby("Group")["Member"].count().sort_values()
    
    #Creating a set of group values which are travelling with more than 1 members.
    y = set(x[x>1].index)
    
    #Creating a new feature "Solo" which will indicate whether the person is travelling solo or not.
    df["Travelling_Solo"] = df["Group"].apply(lambda x: x not in y)
    
    #Creating a new feature "Group_size" which will indicate each group number of members.
    df["Group_Size"]=0
    for i in x.items():
        df.loc[df["Group"]==i[0],"Group_Size"]=i[1]

In [5]:
train_clean= passengerid_new_features(train_raw)
test_clean=passengerid_new_features(test_raw)